<a href="https://colab.research.google.com/github/leobioinf0/sequentia-biotech/blob/main/Ex1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Exercise 1

using the attached file `Variants_file.vcf.gz`


### 1. Which program was used to generate the `VCF` file? Describe how you obtained this information.

We download the file.

In [1]:
!wget https://raw.githubusercontent.com/leobioinf0/sequentia-biotech/main/Variants_file.vcf.gz

--2021-10-13 14:20:43--  https://raw.githubusercontent.com/leobioinf0/sequentia-biotech/main/Variants_file.vcf.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2595852 (2.5M) [application/octet-stream]
Saving to: ‘Variants_file.vcf.gz’

Variants_file.vcf.g 100%[===================>]   2.48M  --.-KB/s    in 0.07s   

2021-10-13 14:20:44 (37.2 MB/s) - ‘Variants_file.vcf.gz’ saved [2595852/2595852]



We will use the "PyVCF" module.

In [2]:
# PyVCF installation
!pip install PyVCF

  Created wheel for PyVCF: filename=PyVCF-0.6.8-cp37-cp37m-linux_x86_64.whl size=124100 sha256=b41d8d000d7bd31839f27a5afac517c079cc12314083ecf52fb7e645253e4186
  Stored in directory: /root/.cache/pip/wheels/8a/60/8d/fe98f8401a0bb73b44afbbc454d1d7fe11a0a2081f5b899597
Successfully built PyVCF


We import the necessary modules.

In [3]:
import vcf
from IPython.display import IFrame
import pandas as pd
%load_ext rpy2.ipython

We read the file "Variants_file.vcf.gz".

In [4]:
vcf_reader = vcf.Reader(filename = "Variants_file.vcf.gz", compressed = True)

we can obtain the program that has been used by reading the metadata source.

In [5]:
src = vcf_reader.metadata["source"][0]
print("VCF file generated by:\t{}".format(src))

VCF file generated by:	freeBayes v1.3.2-dirty


### 2. What is the size of the reference genome used for the analysis? Describe how you calculated it.

We can obtain an estimate of the size of the genome by summing the length of all the contigs.

In [6]:
total = sum(contig.length for contig in vcf_reader.contigs.values())/10**6
print("total length (Mb): {}".format(round(total,2)))

total length (Mb): 687.46


If we assume that Xiphias gladius (assembly ASM1685928v1) was used, we can check its size in the ncbi database.

[Xiphias gladius (assembly ASM1685928v1)](https://www.ncbi.nlm.nih.gov/genome/8987)

total length (Mb): 691.78

In [7]:
IFrame(src="https://www.ncbi.nlm.nih.gov/genome/8987", width=700, height=600)

### 3. Generate a table with the percentage of missing values for each sample.


we will use the bcftools program to obtain stats

In [8]:
!apt-get install -y bcftools

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libhts2
Suggested packages:
  python-matplotlib texlive-latex-recommended
The following NEW packages will be installed:
  bcftools libhts2
0 upgraded, 2 newly installed, 0 to remove and 37 not upgraded.
Need to get 802 kB of archives.
After this operation, 2,426 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libhts2 amd64 1.7-2 [300 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 bcftools amd64 1.7-2 [503 kB]
Fetched 802 kB in 1s (941 kB/s)
Selecting previously unselected package libhts2:amd64.
(Reading database ... 155047 files and directories currently installed.)
Preparing to unpack .../libhts2_1.7-2_amd64.deb ...
Unpacking libhts2:amd64 (1.7-2) ...
Selecting previously unselected package bcftools.
Preparing to unpack .../bcftools_1.7-2_amd64.deb ...
Unpacking bc

In [9]:
!bcftools stats -s - Variants_file.vcf.gz > Variants_file_stats.vchk

we parse Variants_file_stats.vchk to get the total number of records

In [11]:
with open('Variants_file_stats.vchk') as f:
     n_records = [int(line.split(sep="\t")[-1].strip()) for line in f.readlines() if 'number of records:' in line][0]

n_records

4113

We obtain the total number of missings for each sample

In [14]:
df=pd.read_csv("Variants_file_stats.vchk",
               sep="\t",
               skiprows=402,
               nrows=len(vcf_reader.samples),
               usecols=['[3]sample','[14]nMissing'])
df

,[3]sample,[14]nMissing
0,36-CT180742-A01-D05,13
1,184-CT180920-B01-H11,9
2,182-CT180680-B01-F11,27
3,181-CT180337-B01-E11,58
4,73-CT182941-A01-A10,37
...,...,...
78,53-CT182006-A01-E07,87
79,52-CT181983-A01-D07,61
80,54-CT182121-A01-F07,61
81,189-CT182642-B01-E12,71


We calculate the percentage

In [15]:
df["[14]nMissing"] = df["[14]nMissing"].apply(lambda x: x/n_records*100)
df.rename(columns={"[14]nMissing": "missing_pct", '[3]sample':'sample'}, inplace=True)
df

,sample,missing_pct
0,36-CT180742-A01-D05,0.316071
1,184-CT180920-B01-H11,0.218818
2,182-CT180680-B01-F11,0.656455
3,181-CT180337-B01-E11,1.410163
4,73-CT182941-A01-A10,0.899587
...,...,...
78,53-CT182006-A01-E07,2.115244
79,52-CT181983-A01-D07,1.483102
80,54-CT182121-A01-F07,1.483102
81,189-CT182642-B01-E12,1.726234


we save the results in a file

In [16]:
df.to_csv("missing_pct.tsv",sep= "\t")

### 4. Show the clustering of the samples based on the variants included in the `VCF`. Describe the method you have chosen and justify it.

Unzip the file using gunzip

In [17]:
!gunzip -c Variants_file.vcf.gz >  Variants_file.vcf

In the following steps we will use code in R.

We install the necessary libraries, It will take a few minutes:

In [18]:
%%R
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install(c("gdsfmt","SNPRelate"))

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/BiocManager_1.30.16.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 262502 bytes (256 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =


Update all/some/none? [a/s/n]: n


In [19]:
%%R
library(ggplot2)
library(gdsfmt)
library(SNPRelate)

R[write to console]: SNPRelate -- supported by Streaming SIMD Extensions 2 (SSE2)



Parsing

The next command will turn the VCF file into a less data intensive form (GDS) for easier computing.

In [20]:
%%R
snpgdsVCF2GDS("Variants_file.vcf","Variants_file.gds",method ="biallelic.only")

Start file conversion from VCF to SNP GDS ...
Method: extracting biallelic SNPs
Number of samples: 83
Parsing "Variants_file.vcf" ...
	import 4113 variants.
+ genotype   { Bit2 83x4113, 83.3K } *
Optimize the access efficiency ...
Clean up the fragments of GDS file:
    open the file 'Variants_file.gds' (119.1K)
    # of fragments: 46
    save to 'Variants_file.gds.tmp'
    rename 'Variants_file.gds.tmp' (118.8K, reduced: 312B)
    # of fragments: 20


Formatting for IBS matrix

we prepare the data so it is formatted correctly to create a Identity by State matrix (fraction of identity by state for each pair of samples).

In [21]:
%%R
# Open a GDS file
genofile<-snpgdsOpen("Variants_file.gds")

Identity-By-State (IBS) analysis on genotypes:
Excluding 177 SNPs (monomorphic: TRUE, MAF: NaN, missing rate: NaN)
    # of samples: 83
    # of SNPs: 3,936
    using 2 threads
IBS:    the sum of all selected genotypes (0,1,2) = 492905
Wed Oct 13 14:38:04 2021    (internal increment: 65536)
[==================================================] 100%, completed, 0s
Wed Oct 13 14:38:04 2021    Done.


We perform cluster analysis on the n×n matrix of genome-wide IBS pairwise distances, and determine the groups by a permutation score:

In [ ]:
set.seed(100)
ibs.hc<-snpgdsHCluster(snpgdsIBS(genofile,num.thread=2, autosome.only=FALSE))

Turn the clustering into a tree file and plotting tree.

This step takes the clustering results from before and turns the numerical values into a dendrogram

In [34]:
%%R
# Determine groups of individuals automatically
rv <- snpgdsCutTree(ibs.hc)

Determine groups by permutation (Z threshold: 15, outlier threshold: 5):
Create 1 groups.


plot dendogram

In [23]:
%%R
pdf(file = "Dendrogram_based_on_IBS.pdf", width = 15,height = 15)
plot(rv$dendrogram,main="Dendrogram based on IBS", )
dev.off()

png 
  2 


Dissimilarity matrices

This command creates an dissimilarity matrix between all the samples.


Clustering Analysis


This step performs a clustering analysis similar to the IBS analysis above but using dissimilarity. The next line creates a tree file based on dissimilarity rather than relatedness.

In [24]:
%%R
# Calculate the individual dissimilarities for each pair of individuals.
dissMatrix  <-  snpgdsDiss(genofile , 
                          sample.id=NULL, 
                          autosome.only=FALSE,
                          remove.monosnp=TRUE,  
                          maf=NaN, 
                          missing.rate=NaN, 
                          num.thread=2, 
                          verbose=TRUE)

Individual dissimilarity analysis on genotypes:
Excluding 177 SNPs (monomorphic: TRUE, MAF: NaN, missing rate: NaN)
    # of samples: 83
    # of SNPs: 3,936
    using 2 threads
Dissimilarity:    the sum of all selected genotypes (0,1,2) = 492905
Dissimilarity:	Wed Oct 13 14:40:10 2021	0%
Dissimilarity:	Wed Oct 13 14:40:10 2021	100%


Perform hierarchical cluster analysis on the dissimilarity matrix.

In [25]:
%%R
snpHCluster =  snpgdsHCluster(dissMatrix, 
                              sample.id=NULL, 
                              need.mat=TRUE, 
                              hang=0.01)

To determine sub groups of individuals using a specified dendrogram from hierarchical cluster analysis

In [26]:
%%R
cutTree = snpgdsCutTree(snpHCluster, verbose=TRUE)

Determine groups by permutation (Z threshold: 15, outlier threshold: 5):
Create 1 groups.


In [27]:
%%R
snpgdsClose(genofile)

Displaying a tree based on dissimilarity

In [29]:
%%R
pdf(file = "Dendrogram_based_on_dissimilarity.pdf", width = 15,height = 15)
snpgdsDrawTree(cutTree, 
               main = "Dendrogram based on dissimilarity",
               y.label.kinship=T,
               leaflab="perpendicular",
               yaxis.kinship=F)

dev.off()

png 
  2 


### 5. Generate a new `VCF` removing the variants with a `MAF` value higher than 1%. Describe how you generated it.

In [30]:
!apt-get install -y vcftools

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tabix
The following NEW packages will be installed:
  tabix vcftools
0 upgraded, 2 newly installed, 0 to remove and 37 not upgraded.
Need to get 778 kB of archives.
After this operation, 3,192 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tabix amd64 1.7-2 [326 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 vcftools amd64 0.1.15-1 [452 kB]
Fetched 778 kB in 1s (954 kB/s)
Selecting previously unselected package tabix.
(Reading database ... 155094 files and directories currently installed.)
Preparing to unpack .../archives/tabix_1.7-2_amd64.deb ...
Unpacking tabix (1.7-2) ...
Selecting previously unselected package vcftools.
Preparing to unpack .../vcftools_0.1.15-1_amd64.deb ...
Unpacking vcftools (0.1.15-1) ...
Setting up vcftools (0.1.15-1) ...
Setting up tabix (

Include only sites with a Minor Allele Frequency less than or equal to "--max-maf 0.01".

Allele frequency is defined as the number of times an allele appears over all individuals at that site, divided by the total number of non-missing alleles at that site.

In [31]:
#Output a new vcf file from the input vcf file that removes sites with MAF greater than or equal to 0.01
!vcftools --vcf Variants_file.vcf --max-maf 0.01 --recode --out Variants_file_maf


VCFtools - 0.1.15
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--vcf Variants_file.vcf
	--max-maf 0.01
	--out Variants_file_maf
	--recode

After filtering, kept 83 out of 83 Individuals
Outputting VCF file...
After filtering, kept 323 out of a possible 4113 Sites
Run Time = 1.00 seconds


### 6. How many variants are left from the previous step? Describe how you calculated it.

In [32]:
!apt-get install -y sed

Reading package lists... Done
Building dependency tree       
Reading state information... Done
sed is already the newest version (4.4-2).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


`Variants_file_maf.log` tells us the number of variants. We extract the info with "sed"

In [33]:
!sed 'x;$!d' <Variants_file_maf.log

After filtering, kept 323 out of a possible 4113 Sites
